## SQLAlchemy 


Trata-se de uma biblioteca SQL e ORM (Object Relational Mapper) para Python que permite aos desenvolvedores interagir com bancos de dados de uma maneira mais abstrata e orientada a objetos. 

Ela suporta uma variedade de sistemas de banco de dados e facilita o mapeamento entre classes Python e tabelas de banco de dados, além de proporcionar uma maneira de construir consultas SQL de forma programática e segura.

### Usando o SQLAlchemy

É simples e vamos basicamente seguir os seguintes passos:

1. `Instalação dos pacotes necessários`: Instalação do SQLAlchemy.
2. `Configuração da conexão`: Configurar a string de conexão para acessar o banco de dados ( vamos utilizar o Oracle, mas pode ser qualquer banco).
3. `Estabelecer a conexão`: estabelecer a conexão com o banco de dados da nossa preferência.
4. `Executar consultas`: executar consultas SQL usando a conexão estabelecida.
5. `Fechamento da conexão`: fechar a conexão de maneira segura ao final das operações.

In [ ]:
# Instalação dos pacotes necessários
!pip install sqlalchemy oracledb

In [ ]:
# Verificação da versão dos pacotes instalados
import sqlalchemy
import oracledb
print(sqlalchemy.__version__)
print(oracledb.version)

### Nossa tabela de exemplo

Já temos uma tabela criada em nosso banco de dados chamada `PETSHOP`:

```SQL
table: petshop
schema:
    id  -- PK number
    tipo_pet -- varchar 30
    nome_pet -- varchar 30
    idade -- number int
```

Tenha certeza de que a tabela existe.

In [9]:
# Importação das nossas credenciais
import json

with open("credenciais.json") as f:
    credenciais = json.load(f)

USER = credenciais["user"]
PASS = credenciais["pass"]

HOST = "oracle.fiap.com.br"
PORT = 1521
SID = "ORCL"


In [2]:
# Configuração da conexão
from sqlalchemy import create_engine

# Substitua 'USER', 'PASS', 'HOST', 'PORT', e 'SID' pelos seus dados de conexão
connection_string = f"oracle+oracledb://{USER}:{PASS}@{HOST}:{PORT}/{SID}"

# Criar engine de conexão
engine = create_engine(connection_string)

# Estabelecer conexão
connection = engine.connect()

In [3]:
# Podemos fazer as consultas diretamente em SQL
# O uso do text() informa ao SQLAlchemy que a string deve ser tratada como uma instrução SQL textual.

from sqlalchemy import text

# Executar uma consulta e imprimir o resultado
result = connection.execute(text("SELECT * FROM PETSHOP"))
for row in result:
    print(row)

# Fechar conexão
connection.close()

(63, 'Cachorro', 'Rex', 5)
(64, 'Cachorro', 'toto', 15)
(44, 'tigre', 'bengala', 12)
(62, 'Cachorro', 'Rex', 5)
(33, 'elefante', 'jurandir', 6)
(34, 'elefante', 'jurandir', 2000)
(35, 'elefante', 'jurandir', 2000)
(36, 'elefante', 'jurandir', 2000)
(37, 'elefante', 'jurandir', 2000)
(38, 'elefante', 'jurandir', 2000)
(39, 'leao', 'pedroca', 20)
(40, 'leao', 'pedroca', 20)
(41, 'leao', 'pedroca', 20)
(42, 'leao', 'pedroca', 20)
(43, 'leao', 'pedroca', 20)
(5, 'cachoro', 'pituca', 3)
(6, 'cachoro', 'pituca', 3)
(7, 'cachoro', 'pituca', 3)
(8, 'cachoro', 'pituca', 3)


### ORM (Object-Relational Mapping)

Podemos consultar dados de um banco de dados Oracle usando SQLAlchemy sem escrever diretamente as consultas SQL, utilizando a abordagem de ORM (Object-Relational Mapping) que o SQLAlchemy fornece. 

Isso permite que você interaja com o banco de dados usando objetos Python, o que pode ser mais intuitivo e seguro contra injeções SQL. 

Vamos entender como utilizar essa abordagem:

1. `Definir modelos de dados`: Primeiro, você define `classes Python` que mapeiam para as tabelas do banco de dados. Cada classe corresponderá a uma tabela, e cada instância da classe a uma linha na tabela.

2. `Criar uma sessão`: A sessão no SQLAlchemy permite agrupar várias operações em uma única transação.

3. `Consultar dados usando a sessão`: Você pode usar métodos como query() para buscar dados, filtrar resultados, ordenar, etc., tudo sem escrever SQL explicitamente.


In [ ]:
# Definindo o Modelo de Dados
# Primeiro, você define uma classe que mapeia para a tabela petshop no banco de dados. 
# Atenção: Cada atributo da classe corresponderá a uma coluna na tabela.

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base


# Base para os modelos de dados
Base = declarative_base()

# Definição do modelo para a tabela petshop
class Petshop(Base):
    __tablename__ = 'petshop'
    id = Column(Integer, primary_key=True)
    tipo_pet = Column(String(30))
    nome_pet = Column(String(30))
    idade = Column(Integer)


- `Base`: É uma classe base do SQLAlchemy de onde todos os modelos herdam, usada para definir a estrutura do banco de dados em termos de objetos Python.
- `__tablename__` define o nome da tabela no banco de dados.
- `Column` define uma coluna na tabela. O primeiro argumento é o tipo de dado da coluna, e os argumentos nomeados são configurações adicionais como `primary_key`.

- `Session`: Usado para criar uma instância de sessão que permite você fazer queries e outras transações no banco de dados.


In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

# Configuração da conexão
# Substitua 'USER', 'PASS', 'HOST', 'PORT', e 'SID' pelos seus dados de conexão
connection_string = f"oracle+oracledb://{USER}:{PASS}@{HOST}:{PORT}/{SID}"

# Criar engine de conexão
engine = create_engine(connection_string)

# sessionmaker é uma fábrica de sessões, que é usada para criar sessões de banco de dados
Session = sessionmaker(bind=engine)

# Criar uma sessão
session = Session()

# Consultar dados
pets = session.query(Petshop).all()  # Obtém todos os registros da tabela PETSHOP

for pet in pets:
    print(f"{pet.nome_pet}, {pet.tipo_pet}, {pet.idade} anos")  # Acessar os atributos da cada pet

# Fechar sessão
session.close()

### Operações CRUD com SQLAlchemy

Vamos incluir operações básicas de CRUD (Create, Read, Update, Delete) usando SQLAlchemy. 

Isso permitirá que você manipule dados na tabela petshop de várias maneiras. :)

In [ ]:
# CREATE: Adicionar um novo pet

# Create: Um novo objeto Petshop é criado e adicionado à sessão com session.add(), 
# e então as mudanças são salvas no banco de dados com session.commit()`.

novo_pet = Petshop(tipo_pet='Cachorro', nome_pet='toto', idade=15)
session.add(novo_pet)
session.commit()

In [ ]:
# READ: Consultar dados
pets = session.query(Petshop).all()
for pet in pets:
    print(f"{pet.nome_pet}, {pet.tipo_pet}, {pet.idade} anos")

In [ ]:
# UPDATE: Atualizar dados de um pet existente

# Update: Encontra um registro específico (neste caso, pelo nome), 
# altera um atributo e salva a alteração no banco de dados.

pet_a_atualizar = session.query(Petshop).filter(Petshop.nome_pet == 'jurandir').first()

if pet_a_atualizar:
    pet_a_atualizar.idade = 6
    session.commit()

In [ ]:
# DELETE: Deletar um pet

# Delete: Encontra um registro específico (neste caso, pelo nome),
# e o remove do banco de dados.

pet_a_deletar = session.query(Petshop).filter(Petshop.nome_pet == 'loro jose').first()

if pet_a_deletar:
    session.delete(pet_a_deletar)
    session.commit()

In [ ]:
# Fechar sessão

# O método `session.close()` encerra a conexão com o banco de dados e libera recursos.

session.close()

***Dica*** Quando usar o session.close()

É uma boa prática fechar a sessão quando você termina todas as operações que planeja executar nessa sessão específica.

- `Após uma série de operações`: Se você está executando uma série de operações CRUD (criar, ler, atualizar, deletar) e elas são todas parte de um único bloco lógico de trabalho, é adequado fechar a sessão após essas operações estarem completas.

- `Em scripts ou funções autônomas`: Se o seu código está estruturado de forma que as operações de banco de dados estão contidas em uma função ou script que é executado e depois termina, você deve fechar a sessão ao final dessas operações.

- `Antes de lançar exceções`: Se você detectar erros e planeja lançar uma exceção que pode terminar o script ou mudar o fluxo do programa, é uma boa prática fechar a sessão antes de lançar a exceção para garantir que os recursos sejam liberados corretamente.

### Básico da `session.query()`


Nos exemplos acima realiamos alguma 'query', vamos entender e explorar mais algumas possiblidades: 

'session.query()' é usado para criar um objeto de consulta que permite selecionar dados de uma ou mais tabelas. Vamos começar com alguns exemplos básicos e então explorar algumas funcionalidades mais avançadas.

#### Selecionar Todos os Registros

Este é o exemplo mais simples, onde selecionamos todos os registros de uma tabela.

In [ ]:
#### lembre de criar a sessão novamente para poder fazer a consulta

# Selecionar todos os registros da tabela Petshop
todos_pets = session.query(Petshop).all()

for pet in todos_pets:
    print(f"{pet.nome_pet}, {pet.tipo_pet}, {pet.idade} anos")


#### Filtrar Registros

Você pode usar o método `filter()` para especificar condições que os registros devem satisfazer para serem incluídos no resultado.

In [ ]:
# Selecionar todos os pets que são cachorros
cachorros = session.query(Petshop).filter(Petshop.tipo_pet == 'cachoro').all()
for cachorro in cachorros:
    print(f"{cachorro.nome_pet}, {cachorro.idade} anos")


#### Ordenar Resultados

Use o método `order_by()` para ordenar os resultados da consulta.

In [ ]:
# Ordenar pets por idade
pets_ordenados = session.query(Petshop).order_by(Petshop.idade).all()
for pet in pets_ordenados:
    print(f"{pet.nome_pet}, {pet.tipo_pet}, {pet.idade} anos")


#### Limitar Resultados

Para limitar o número de resultados retornados, use o método `limit()`.

In [ ]:
# Selecionar os três primeiros pets
primeiros_tres_pets = session.query(Petshop).limit(3).all()
for pet in primeiros_tres_pets:
    print(f"{pet.nome_pet}, {pet.tipo_pet}, {pet.idade} anos")


#### Contar Registros

Para contar o número de registros que atendem a uma condição, use o método `count()`.


In [ ]:
# Contar quantos cachorros existem na tabela
quantidade_cachorros = session.query(Petshop).filter(Petshop.tipo_pet == 'leao').count()
print(f"Quantidade de leao: {quantidade_cachorros}")


In [ ]:
# alternativa com filter_by(), é mais simples mas menos flexível

quantidade = session.query(Petshop).filter_by(tipo_pet='leao').all()
print(f"Quantidade de leao: {len(quantidade)}")

#### Selecionar Colunas Específicas

Se você quiser selecionar apenas algumas colunas específicas em vez de objetos completos, você pode especificar isso na chamada do `query()`.

In [ ]:
# Selecionar apenas o nome e idade dos pets
nomes_idades = session.query(Petshop.nome_pet, Petshop.idade).all()
for nome, idade in nomes_idades:
    print(f"{nome}, {idade} anos")


### Criando uma nova tabela no banco de dados

Para criar uma nova tabela em um banco de dados usando SQLAlchemy, vamos definir a estrutura da tabela em forma de uma `classe Python`, utilizando a base declarativa do SQLAlchemy. Depois, você pode usar esta definição para efetivamente criar a tabela no banco de dados.

- Passo 1: Definir a Classe da Tabela
- Passo 2: Criar a Tabela no Banco de Dados
- Passo 3: Usar a nova Tabela :)

#### Nossa nova tabela de exemplo

Vamos criar em nosso banco de dados uma tabela chamada `Clientes`:

```SQL
table: Clientes
schema:
    id  -- PK number
    nome -- varchar 100
    email -- varchar 100
    saldo -- number float
```

#### Passo 1: Definir a Classe da Tabela

Primeiramente, definimos a classe que mapeia para a tabela no banco de dados. Essa classe herda de Base, que é uma instância criada pela função `declarative_base()` do SQLAlchemy.

In [40]:
from sqlalchemy import Sequence, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

# Base para os modelos de dados
Base = declarative_base()

# Definição do modelo para a tabela clientes
class Cliente(Base):
    __tablename__ = 'clientes'
    id = Column(Integer, Sequence('cliente_id_seq'), primary_key=True)
    nome = Column(String(100), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    saldo = Column(Float, default=0.0)

# FAZER A CONEXÃO COM O BANCO DE DADOS

# ....
# ....
# ....

#### Passo 2: Criar a Tabela no Banco de Dados

Depois de definir a classe e configurar a conexão com banco, você pode criar a tabela no banco de dados chamando o método `create_all()` do objeto Base, passando a engine como argumento. 

Este método irá verificar todas as classes que herdam de Base e criar as tabelas no banco de dados conforme necessário, se ela ainda não existir. Se a tabela já existir e você tiver modificado sua definição, você precisará atualizar a tabela manualmente ou recriá-la.

In [41]:
# Criar todas as tabelas definidas
Base.metadata.create_all(engine)

print("Tabela criada com sucesso!")

#### dica extra: como recriar a tabela
# Para recriar a tabela, garanta que ela seja primeiro excluída
# Cliente.__table__.drop(engine, checkfirst=True)  # Exclui a tabela se ela existir
# Base.metadata.create_all(engine)                # Cria a tabela


Tabela criada com sucesso!


#### Passo 3: Usar a nova tabela

Podemos usar nossa nova tabela `cliente` usando o que já aprendemos nesta aula.

In [44]:
# Criar uma sessão
Session = sessionmaker(bind=engine)
session = Session()

# Adicionar um novo cliente
novo_cliente = Cliente(nome='gabi',
                       email='gabi@email.com',
                       saldo=100.0)
session.add(novo_cliente)
session.commit()

# Consultar todos os clientes
clientes = session.query(Cliente).all()
for cliente in clientes:
    print(f"{cliente.nome}, {cliente.email}, R$ {cliente.saldo:.2f}")
    
# Fechar sessão
session.close()

João Silva, eid@emple.com, R$ 100.00
gabi, gabi@email.com, R$ 100.00
Arnaldo, arnaldo@email.com, R$ 100.00
joaquinzinho, jojo@email.com, R$ 100.00
